# **Rijndael (AES) Encryption**
## *Marta Llopart  & Pol Medina*

In [1]:
# CREATED BY: Pol Medina and Marta Llopart
# Data Security Lab - Group 1

import numpy as np

### Create a function that given a text string creates a matrix of 4x4 bytes.

In [2]:
def create_string_matrix(message:str, string:bool=True, padding:bool=True) -> np.ndarray:
    """
    Given a plaintext message, encode it into an hexadecimal matrix.

    Parameters
    ----------
    message : str
        The string message to encode.

    Returns
    -------
    matrix : numpy.ndarray
        A 4x4 numpy array (np.ndarray) containing the hexadecimal encoding of the message.
    """
    if padding:
        while len(message) < 16:
            message += '0'
    if string:
        message = message.encode("utf-8").hex()
    m = ""
    hex_array = np.array([])
    int_array = np.array([])

    while message:
        for i in range(2):
            m += message[i]

        message = message[2:]
        hex_array = np.append(hex_array, m)
        int_array = np.append(int_array, int(m, 16))
        m = ""
        
    hex_array = hex_array.reshape(4, 4)
    int_array = int_array.reshape(4, 4).astype(int)

    return np.transpose(int_array), np.transpose(hex_array)

### Create the functions that convert from hexadecimal to binary and from binary to hexadecimal.

In [3]:
def hex_to_bin(ini_string, scale=16):
    # return bin(int(ini_string, scale)).zfill(8) 
    return bin(int(ini_string, 16))[2:]

def bin_to_hex(binary):
    return hex(int(binary, 2))

### Create the functions that allow the computation of the MixColumns step.

In [4]:
def get_constant_matrix() -> np.ndarray:
    """
    Returns a constant matrix of 4x4
    """
    return np.array([[2, 3, 1, 1],
	   		  	 	 [1, 2, 3, 1],
					 [1, 1, 2, 3],
					 [3, 1, 1, 2]])


def compute_modulo(a: int, b: bin) -> bin:
	"""
	Given two inputs, it computes the multiplication between them

	Parameters
	----------
	number : int
	    The integer number to multiply with binary [1, 3].
	binary : hex
	    The hexadecimal number to multiply with integer.

	Returns
	-------
    binary : bin
	    A binary number that results from the multiplication.
	"""

	output = None
	constant = 0b100011011
	
	b = int(hex_to_bin(b), 2)

	if a == 2:
		b = b << 1
		if b >= 0b100000000:
			output = b ^ constant
		else:
			output = b
	elif a == 3:
		z = b << 1 # move 1 to left
		b = b ^ z # XOR (sum) z + b = x*b + b
		if b >= 0b100000000:
			output = b ^ constant
		else:
			output = b
	elif a == 1:
		return b
	return output


def mutliply_row_by_column(row: np.ndarray, column:np.ndarray) -> bin:
	"""
	Multiplies the row of a matrix with the
	column of another matrix.

	Parameters
	----------
    row : np.ndarray
	    The row to multiply.
	column : np.ndarray
	    The column to multiply.
		
	Returns
	-------
    binary : bin
	    The binary number that results from the multiplication.
	"""
	output = []

	# [a, b, c, d] * [e, f, g, h]
	# output: [a*e + b*f + c*g + d*h]
	for i in range(len(row)):
		output.append(compute_modulo(row[i], column[i])) # row[i] * column[i] 
	result = output[0] ^ output[1] ^ output[2] ^ output[3]
	return result


def mix_columns(constant_matrix: np.ndarray, substituted_matrix: np.ndarray) -> np.ndarray:
	"""
	Given two different matrices, it multiplies them.
	
    Parameters
	----------
    constant_matrix : np.ndarray
        The constant matrix.
	substituted_matrix : np.ndarray
        The plaintext matrix.
	
	Returns
	-------
    matrix : np.ndarray
        The resulting NxN matrix.
	"""
	# he de hacer row por row y transponer el resultado
	hex_result,	bin_result, result = [], [], []
	substituted_matrix = np.transpose(substituted_matrix)

	for i in substituted_matrix:
		hex_row, bin_row, tmp_row = [], [], []
		for j in constant_matrix:
			value = mutliply_row_by_column(j, i)
			hex_row.append(hex(value))
			bin_row.append(bin(value))
			tmp_row.append(value)
		result.append(tmp_row)
		hex_result.append(hex_row)
		bin_result.append(bin_row)
	
	return [np.transpose(np.array(result)),
		    np.transpose(np.array(hex_result)),
		    np.transpose(np.array(bin_result))]

### Create the functions that allow the computation of the ShiftRows step.

In [5]:
def shift_rows(matrix):    
    out_hex = matrix.copy()

    for i in range(len(out_hex)):
        out_hex[i] = out_hex[i][i:] + out_hex[i][:i]

    out_int = []
    for row in out_hex:
        out_int.append([int(x, 16) for x in row])

    return np.array(out_int), np.array(out_hex)

### Create the functions that allow the computation of the SubBytes step.

In [6]:
def get_sbox_value(hex_num, item_t = 'hex'):
    matrix = [[0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76],
              [0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0],
              [0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15],
              [0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75],
              [0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84],
              [0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf],
              [0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8],
              [0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2],
              [0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73],
              [0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb],
              [0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79],
              [0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08],
              [0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a],
              [0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e],
              [0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf],
              [0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16]]

    if item_t == 'str':
        for i, row in enumerate(matrix):
                matrix[i] = [hex(x) for x in row]

    if len(hex_num) == 3:
        row = 0
    else:   
        row = int(str(hex_num)[-2], 16)
        
    col = int(str(hex_num)[-1], 16)

    return matrix[row][col]


def substitute_bytes(matrix):
    out = matrix.copy()

    for i, row in enumerate(matrix):
        out[i] = [get_sbox_value(hex(x), 'str') for x in row]
    return out

### Create the functions that allow the computation of the AddRoundKey step.

In [7]:
def add_round_key(message_matrix, round_key, verbose=False):
    out_matrix = []
    for i in range(len(message_matrix)):
        out_row = []
        for j, item in enumerate(message_matrix[i]):
            out_row.append(item ^ round_key[i][j])
        out_matrix.append(out_row)

    copy = out_matrix.copy()
    for i, row in enumerate(copy):
        print
        copy[i] = [hex(x) for x in row]

    if verbose:
        print(f"Input message matrix for AddRoundKey:")
        print(" ------------------------------- ")
        for row in message_matrix:
            print([hex(x) for x in row])
        print(" ------------------------------- ")

        print(f"Input round key matrix for AddRoundKey:")
        print(" ------------------------------- ")
        for row in round_key:
            print([hex(x) for x in row])
        print(" ------------------------------- ")

        print(f"Output matrix for AddRoundKey:")
        print(" ------------------------------- ")
        for row in copy:
            print(row)
        print(" ------------------------------- ")
    
    return out_matrix, copy

### Create the functions that allow the computation of the round keys.

In [8]:
def get_rcon_matrix():
    return np.transpose(
           np.array([[0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40, 0x80, 0x1b, 0x36],
                     [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
                     [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
                     [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0]]))


def substituted_final_column(matrix):
    cipher_key = matrix.copy()
    rot_word = cipher_key[-1][1:].tolist()
    rot_word.append(cipher_key[-1][0])

    cipher_key[-1] = np.array(rot_word)
    cipher_key = cipher_key.tolist()

    rotated_word = substitute_bytes(cipher_key)[-1]

    for i, s in enumerate(rotated_word):
        rotated_word[i] = int(s, 16)
    
    return rotated_word


def generate_seed(key_column, seed_column, rcon_column):
    out_vector = []
    for i, item in enumerate(seed_column):
        result = item ^ key_column[i] ^ rcon_column[i]
        out_vector.append(hex(result))
    return out_vector


def generate_column(key_column, seed_column):
    out_vector = []
    for i, item in enumerate(seed_column):
        result = item ^ key_column[i]
        out_vector.append(hex(result))
    return out_vector


def generate_round_matrix(input_key, rcon_matrix, nRound):
    output_matrix = []
    key = input_key.copy()
    initial_seed = substituted_final_column(key)
    for i in range(len(input_key)):
        if i == 0:
            column = generate_seed(key[i], initial_seed, rcon_matrix[nRound-1])
            column = [int(hexadecimal, 16) for hexadecimal in column]
            output_matrix.append(column)
        else:
            column = generate_column(key[i], output_matrix[-1])
            column = [int(hexadecimal, 16) for hexadecimal in column]
            output_matrix.append(column)
    return np.array(output_matrix)


def get_round_key_matrices(original_key, nRounds=10):
    state_key = original_key.copy()
    state_key = np.transpose(state_key)
    Rcon = get_rcon_matrix()
    key_list = [state_key]
    for nRound in range(1, nRounds+1):
        output = generate_round_matrix(key_list[-1], Rcon, nRound)
        key_list.append(output)
    output_keys = [np.transpose(key) for key in key_list]
    return output_keys

### Create the function that allows the computation of the whole round pipeline.

In [9]:
def complete_round_step(init_matrix, key_list, nRound, verbose=False):
    constant_matrix = get_constant_matrix()
    substituted_matrix  = substitute_bytes(init_matrix)   # correct output
    shifted_rows_matrix_int, shifted_rows_matrix_hex = shift_rows(substituted_matrix)  # correct output
    out_int, out_hex, _ = mix_columns(constant_matrix, shifted_rows_matrix_hex)
    key = key_list[nRound]
    round_key_matrix_int, round_key_matrix_hex = add_round_key(out_int,key)
    
    if verbose:
        print("===============================")
        print(f"            ROUND {nRound}        ")
        print("===============================")


        print(f"Input matrix for SubBytes:")
        print(" ------------------------------- ")
        for row in init_matrix:
            print([hex(x) for x in row])
        print(" ------------------------------- \n")


        print(f"Output matrix for SubBytes &")
        print(f"Input matrix for ShiftRows:")
        print(" ------------------------------- ")
        for row in substituted_matrix:
            print(row)
        print(" ------------------------------- \n")


        print(f"Output matrix for ShiftRows &")
        print(f"Input matrix for MixColumns:")
        print(" ------------------------------- ")
        for row in shifted_rows_matrix_hex:
            print(row)
        print(" ------------------------------- \n")


        print(f"Output matrix for MixColumns &")
        print(f"Input matrix for AddRoundKey:")
        print(" ------------------------------- ")
        for row in out_hex:
            print(row)
        print(" ------------------------------- \n")


        print(f"Output matrix for AddRoundKey:")
        print(" ------------------------------- ")
        for row in round_key_matrix_hex:
            print(row)
        print(" ------------------------------- \n\n")

    return round_key_matrix_int, round_key_matrix_hex

# Whole encryption pipeline example showing each step's output

### Step One:
Set key schedule and message and add round key with only the message and the key

In [10]:
nRounds = 10 # set number of rounds (keys)

int_message = np.array([[0x32, 0x88, 0x31, 0xe0],
                        [0x43, 0x5a, 0x31, 0x37],
                        [0xf6, 0x30, 0x98, 0x07],
                        [0xa8, 0x8d, 0xa2, 0x34]])
  
original_key = np.array([[0x2b, 0x28, 0xab, 0x09],
                         [0x7e, 0xae, 0xf7, 0xcf],
                         [0x15, 0xd2, 0x15, 0x4f],
                         [0x16, 0xa6, 0x88, 0x3c]])

key_list = get_round_key_matrices(original_key)

init_int, init_hex = add_round_key(int_message, key_list[0], verbose=True)

Input message matrix for AddRoundKey:
 ------------------------------- 
['0x32', '0x88', '0x31', '0xe0']
['0x43', '0x5a', '0x31', '0x37']
['0xf6', '0x30', '0x98', '0x7']
['0xa8', '0x8d', '0xa2', '0x34']
 ------------------------------- 
Input round key matrix for AddRoundKey:
 ------------------------------- 
['0x2b', '0x28', '0xab', '0x9']
['0x7e', '0xae', '0xf7', '0xcf']
['0x15', '0xd2', '0x15', '0x4f']
['0x16', '0xa6', '0x88', '0x3c']
 ------------------------------- 
Output matrix for AddRoundKey:
 ------------------------------- 
['0x19', '0xa0', '0x9a', '0xe9']
['0x3d', '0xf4', '0xc6', '0xf8']
['0xe3', '0xe2', '0x8d', '0x48']
['0xbe', '0x2b', '0x2a', '0x8']
 ------------------------------- 


### Step Two:
Loop through all the rounds except first and last one

In [11]:
next_round_int = init_int
for round in range(1, nRounds):
    next_round_int, next_round_hex = complete_round_step(next_round_int, 
                                                         key_list, 
                                                         round, 
                                                         verbose=True)

            ROUND 1        
Input matrix for SubBytes:
 ------------------------------- 
['0x19', '0xa0', '0x9a', '0xe9']
['0x3d', '0xf4', '0xc6', '0xf8']
['0xe3', '0xe2', '0x8d', '0x48']
['0xbe', '0x2b', '0x2a', '0x8']
 ------------------------------- 

Output matrix for SubBytes &
Input matrix for ShiftRows:
 ------------------------------- 
['0xd4', '0xe0', '0xb8', '0x1e']
['0x27', '0xbf', '0xb4', '0x41']
['0x11', '0x98', '0x5d', '0x52']
['0xae', '0xf1', '0xe5', '0x30']
 ------------------------------- 

Output matrix for ShiftRows &
Input matrix for MixColumns:
 ------------------------------- 
['0xd4' '0xe0' '0xb8' '0x1e']
['0xbf' '0xb4' '0x41' '0x27']
['0x5d' '0x52' '0x11' '0x98']
['0x30' '0xae' '0xf1' '0xe5']
 ------------------------------- 

Output matrix for MixColumns &
Input matrix for AddRoundKey:
 ------------------------------- 
['0x4' '0xe0' '0x48' '0x28']
['0x66' '0xcb' '0xf8' '0x6']
['0x81' '0x19' '0xd3' '0x26']
['0xe5' '0x9a' '0x7a' '0x4c']
 -------------------------

### Step Three:
Apply substitute bytes, shift rows and add round key to last iteration

In [12]:
substituted_matrix = substitute_bytes(next_round_int)
shifted_matrix_int, shifted_matrix_hex = shift_rows(substituted_matrix)
shifted_matrix_hex

final_int, final_hex = add_round_key(shifted_matrix_int, key_list[-1], verbose=True)
final_hex 

Input message matrix for AddRoundKey:
 ------------------------------- 
['0xe9', '0xcb', '0x3d', '0xaf']
['0x31', '0x32', '0x2e', '0x9']
['0x7d', '0x2c', '0x89', '0x7']
['0xb5', '0x72', '0x5f', '0x94']
 ------------------------------- 
Input round key matrix for AddRoundKey:
 ------------------------------- 
['0xd0', '0xc9', '0xe1', '0xb6']
['0x14', '0xee', '0x3f', '0x63']
['0xf9', '0x25', '0xc', '0xc']
['0xa8', '0x89', '0xc8', '0xa6']
 ------------------------------- 
Output matrix for AddRoundKey:
 ------------------------------- 
['0x39', '0x2', '0xdc', '0x19']
['0x25', '0xdc', '0x11', '0x6a']
['0x84', '0x9', '0x85', '0xb']
['0x1d', '0xfb', '0x97', '0x32']
 ------------------------------- 


[['0x39', '0x2', '0xdc', '0x19'],
 ['0x25', '0xdc', '0x11', '0x6a'],
 ['0x84', '0x9', '0x85', '0xb'],
 ['0x1d', '0xfb', '0x97', '0x32']]

### Encapsulate final function

In [13]:
def encrypt_message(message, key, nRounds, transpose=True):
    key_list = get_round_key_matrices(key)

    init_int, init_hex = add_round_key(message, key_list[0])
    init_hex

    next_round_int = init_int
    for round in range(1, nRounds):
        next_round_int, next_round_hex = complete_round_step(next_round_int, key_list, round)
    
    substituted_matrix = substitute_bytes(next_round_int)
    shifted_matrix_int, shifted_matrix_hex = shift_rows(substituted_matrix)
    shifted_matrix_hex

    final_int, final_hex = add_round_key(shifted_matrix_int, key_list[-1])

    if transpose:
        final_hex = np.transpose(final_hex)

    cipher_chr = []
    cipher_hex = []
    for row in final_hex:
        for item in row:
            if len(item[2:]) < 2:
                cipher_hex.append('0' + item[2:])
            else:
                cipher_hex.append(item[2:])
            cipher_chr.append(chr(int(item, 16)))

    ciphertext_hex = ''.join(cipher_hex)
    ciphertext_string = ''.join(cipher_chr)

    return ciphertext_string, ciphertext_hex, 

encrypted_message_string, encrypted_message_hex = encrypt_message(int_message, original_key, 10)
encrypted_message_hex

'3925841d02dc09fbdc118597196a0b32'

# Test with our message and other examples

In [14]:
# !pip install pycryptodome

In [15]:
from Crypto.Random import get_random_bytes

# Create a random key formed by 16 bytes (hex too)
key = get_random_bytes(16)
key_hex = key.hex()

# Create the message's plaintext and key matrix
m = "POLMEDAMARTALLEN"
int_message, hex_message = create_string_matrix("POLMEDAMARTALLEN")
int_key, hex_key = create_string_matrix(key_hex, string=False)

print("POLMEDAMARTALLEN plaintext in matrix form:")
print(" ---------------------- ")
for row in hex_message.tolist():
    print(row)
print(" ---------------------- \n")

print(f"{key_hex} key in matrix form:")
print(" ---------------------- ")
for row in hex_key.tolist():
    print(row)
print(" ---------------------- ")


# Compute the ciphertext from the plaintext with the key for nrounds = 10
ciphertext_string, ciphertext_hex = encrypt_message(int_message, int_key, 10)
print(f'The ciphertext "{ciphertext_hex}" was created with the key "{key_hex}".\n')

print("==========================================================")
print(f' Plaintext  (str)  : "POLMEDAMARTALLEN"')
print(f' Ciphertext (str)  : "{ciphertext_string}"')
print(f' Ciphertext (hex)  : "{ciphertext_hex}"')
print("==========================================================")

POLMEDAMARTALLEN plaintext in matrix form:
 ---------------------- 
['50', '45', '41', '4c']
['4f', '44', '52', '4c']
['4c', '41', '54', '45']
['4d', '4d', '41', '4e']
 ---------------------- 

da312cad4333e84b55d070e4c296053f key in matrix form:
 ---------------------- 
['da', '43', '55', 'c2']
['31', '33', 'd0', '96']
['2c', 'e8', '70', '05']
['ad', '4b', 'e4', '3f']
 ---------------------- 
The ciphertext "1615d2d677162abc671809de76ff216d" was created with the key "da312cad4333e84b55d070e4c296053f".

 Plaintext  (str)  : "POLMEDAMARTALLEN"
 Ciphertext (str)  : "ÒÖw*¼g	Þvÿ!m"
 Ciphertext (hex)  : "1615d2d677162abc671809de76ff216d"


In [16]:
# Create a random key formed by 16 bytes (hex too)
# key = get_random_bytes(16)
# key_hex = key.hex()

# Create the message's plaintext and key matrix
m = "POLMEDAMARTALLEN"
k = "MYSECRETKEYISPOL"
int_message, hex_message = create_string_matrix("POLMEDAMARTALLEN")
int_key, hex_key = create_string_matrix("MYSECRETKEYISPOL")

print("POLMEDAMARTALLEN plaintext in matrix form:")
print(" ---------------------- ")
for row in hex_message.tolist():
    print(row)
print(" ---------------------- \n")

print(f"{'MYSECRETKEYISPOL'} key in matrix form:")
print(" ---------------------- ")
for row in hex_key.tolist():
    print(row)
print(" ---------------------- ")


# Compute the ciphertext from the plaintext with the key for nrounds = 10
ciphertext_string, ciphertext_hex = encrypt_message(int_message, int_key, 10, True)
print(f'The ciphertext "{ciphertext_hex}" was created with the key "{k.encode("utf-8").hex()}".\n')

print("==========================================================")
print(f' Plaintext  (str)  : "POLMEDAMARTALLEN"')
print(f' Key        (str)  : "MYSECRETKEYISPOL"')
print(f' Key        (hex)  : "{k.encode("utf-8").hex()}"')
print(f' Ciphertext (str)  : "{ciphertext_string}"')
print(f' Ciphertext (hex)  : "{ciphertext_hex}"')
print("==========================================================")

POLMEDAMARTALLEN plaintext in matrix form:
 ---------------------- 
['50', '45', '41', '4c']
['4f', '44', '52', '4c']
['4c', '41', '54', '45']
['4d', '4d', '41', '4e']
 ---------------------- 

MYSECRETKEYISPOL key in matrix form:
 ---------------------- 
['4d', '43', '4b', '53']
['59', '52', '45', '50']
['53', '45', '59', '4f']
['45', '54', '49', '4c']
 ---------------------- 
The ciphertext "8c1f95872e638a7ca08aa1cb9dd33dc9" was created with the key "4d595345435245544b45594953504f4c".

 Plaintext  (str)  : "POLMEDAMARTALLEN"
 Key        (str)  : "MYSECRETKEYISPOL"
 Key        (hex)  : "4d595345435245544b45594953504f4c"
 Ciphertext (str)  : ".c| ¡ËÓ=É"
 Ciphertext (hex)  : "8c1f95872e638a7ca08aa1cb9dd33dc9"


With the page ['Devglan'](https://www.devglan.com/online-tools/aes-encryption-decryption) we can check that the encryption is correct (the web's encryption matches our encryption).

`MESSAGE = POLMEDAMARTALLEN`

`KEY     = MYSECRETKEYISPOL`

![title](img/result.png)

## Exercise 3: compute the ciphertext for each DNI ID.

In [17]:
DNIs = ["06692990J", "67464613R", "46559056H", "66194520Z", "39728532B", "55392382X",
        "72501363G", "95114975Q", "45138027K", "53965875D", "43335814L", "33100590W",
        "33100590W", "31045476F", "87083032H", "92785933T", "18823871G", "42219350K",
        "86008301P", "78701410X", "92609794H", "56553913H", "36205352V", "25858573H",
        "67288319W", "18381276K", "67658584J", "81454311H", "99857356G", "02366378T"]

In [18]:
# Create a random key formed by 16 bytes (hex too)
key = get_random_bytes(16)
key_hex = key.hex()
int_key, hex_key = create_string_matrix(key_hex, string=False)

# Create a list for all DNIs matrices
dni_cipher_list = []
for dni in DNIs:
    int_dni, hex_dni = create_string_matrix(dni)
    ciphertext_dni = encrypt_message(int_dni, int_key, 10)
    dni_cipher_list.append(ciphertext_dni[1])

print(f'There are {len(dni_cipher_list)} DNIs in our small database:')
for dni in dni_cipher_list:
    print(dni)

There are 30 DNIs in our small database:
067450ac5b509949598639dd9768fa8c
417c6b451586e412d285bbf567ac0eae
dd2f0d806487cd5d5fc17116c090f8ed
014328030d4507190496fb6ddd2e75c8
9b5171613c59270de37aabfe30505274
a8a2732be3851e32374c7c960a979655
c70eb5d2fd2cc97879aa5c5a511b5dc6
a7163b018507906aae5fcbd87874b376
dc6d1139ffc61935e66e26500b989784
93015d7e1af5995f6e0b92c31482223a
99e9a914d7e2a263e8b375bc8d5e3fcf
ce1d785662980cafab54cdfefc5bded2
ce1d785662980cafab54cdfefc5bded2
80383d3494cfd469d92c13c9e4b5ca55
ee9be2bad563c04386b49a2c9bae9af2
3f69e45a1560051b958610517ee97d97
ad13d934bd014e0cb8ffc8d11e11ae83
ca6e27f1574bdba2c853c504f4cd1e41
3bb93c05f1f215509930adcca83c2742
408e19eb6e5e47c0ddb12dd948eb6936
c2369631706626eee135e37388867f04
1753748a532e4a3c75437234c132318b
d2a5f8dac604c6af9d0c91150901fd3a
2d0fa8f1ca32ace77a5d0340824929a8
8597943dd09d2a5411bf5ae12554fc9f
9e22072e16c5fe81b46f399817af005a
957201ab226d276ab3432857a3057c83
d582bd429bf4366764b5b130fe8ec116
0c213f2b2b53ba0ef6975e6f252df86f
9a

Compute the correlation between the encrypted DNIs

In [19]:
def hex_to_numeric_vector(hex_strings):
    binary_vectors = []
    
    for hex_string in hex_strings:
        # Convert hex string to an integer
        decimal_value = int(hex_string, 16)
        
        # Convert the integer to its binary representation and format it to a fixed length
        # The length should be a multiple of 4 to fit each hex character (1 hex = 4 bits)
        binary_length = len(hex_string) * 4
        binary_representation = format(decimal_value, '0{}b'.format(binary_length))
        
        # Convert the binary string to a list of integers (0s and 1s)
        numeric_vector = [int(bit) for bit in binary_representation]
        
        binary_vectors.append(numeric_vector)
    
    return binary_vectors

# Convert hex strings to numerical vectors
binary_vectors = hex_to_numeric_vector(dni_cipher_list)

# Print the results
for i, vector in enumerate(binary_vectors):
    print(f"hex: {dni_cipher_list[i]} -> binary: {vector}")

hex: 067450ac5b509949598639dd9768fa8c -> binary: [0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0]
hex: 417c6b451586e412d285bbf567ac0eae -> binary: [0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0]
hex: dd2f0d806487cd5d5fc17116c090f8ed -> binary: [1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0

We will compute the hamming distance, since this computes the number of differing bits between the ciphertexts of same length (128 bits). If they are truly independent, this is no correlated, a high average hamming distance is expected.

In [20]:
# Code obtained through ChatGPT

def hamming_distance(vec1, vec2):
    """Compute the Hamming distance between two binary vectors."""
    return np.sum(np.array(vec1) != np.array(vec2))

def mean_hamming_distance(binary_vectors):
    """Compute the mean Hamming distance between all pairs of binary vectors."""
    n = len(binary_vectors)
    total_distance = 0
    count = 0
    
    # Iterate over all pairs of vectors
    for i in range(n):
        for j in range(i + 1, n):
            total_distance += hamming_distance(binary_vectors[i], binary_vectors[j])
            count += 1
            
    # Calculate the mean Hamming distance
    mean_distance = total_distance / count if count > 0 else 0
    return mean_distance

In [21]:
# Compute the mean Hamming distance
mean_distance = mean_hamming_distance(binary_vectors)
print("Mean Hamming Distance:", mean_distance)

Mean Hamming Distance: 63.864367816091956


For vectors of 128 bits, a hamming distance of 64 means that half of the bits are different, which is the expected value for two random vectors. This is a good measure of independence.